In [1]:
#Keystroke_Manhattan_Filtered_Detector.py
 
from scipy.spatial.distance import cityblock
import numpy as np
np.set_printoptions(suppress = True)
import pandas
from sklearn import metrics
from scipy.spatial.distance import euclidean

#function to evaluate equal error rate (EER) for the detector
def evaluateEER(user_scores, imposter_scores):
    labels = [0]*len(user_scores) + [1]*len(imposter_scores)
    fpr, tpr, thresholds = metrics.roc_curve(labels, user_scores + imposter_scores)
    missrates = 1 - tpr
    farates = fpr
    dists = missrates - farates
    idx1 = np.argmin(dists[dists >= 0])
    idx2 = np.argmax(dists[dists < 0])
    x = [missrates[idx1], farates[idx1]]
    y = [missrates[idx2], farates[idx2]]
    a = ( x[0] - x[1] ) / ( y[1] - x[1] - y[0] + x[0] )
    eer = x[0] + a * ( y[0] - x[0] )
    return eer
 
class ManhattanFilteredDetector:
  
    def __init__(self, subjects):
        self.user_scores = []
        self.imposter_scores = []
        self.mean_vector = []
        self.subjects = subjects
  
    def training(self):
        self.mean_vector = self.train.mean().values
        self.std_vector = self.train.std().values
        dropping_indices = []
        for i in range(self.train.shape[0]):
            cur_score = euclidean(self.train.iloc[i].values, self.mean_vector)
            if (cur_score > 3*self.std_vector).all() == True: dropping_indices.append(i)
        self.train = self.train.drop(self.train.index[dropping_indices])
        self.mean_vector = self.train.mean().values     
  
    def testing(self):
        for i in range(self.test_genuine.shape[0]):
            cur_score = cityblock(self.test_genuine.iloc[i].values, self.mean_vector)
            self.user_scores.append(cur_score)
  
        for i in range(self.test_imposter.shape[0]):
            cur_score = cityblock(self.test_imposter.iloc[i].values, self.mean_vector)
            self.imposter_scores.append(cur_score)
  
    def evaluate(self):
        eers = []
  
        for subject in subjects:        
            genuine_user_data = data.loc[data.subject == subject, "H.period":"H.Return"]
            imposter_data = data.loc[data.subject != subject, :]
             
            self.train = genuine_user_data[:300]
            self.test_genuine = genuine_user_data[300:]
            self.test_imposter = imposter_data.groupby("subject"). head(5).loc[:, "H.period":"H.Return"]
  
            self.training()
            self.testing()
            eers.append(evaluateEER(self.user_scores, self.imposter_scores))
        return np.mean(eers)

#loading of dataset
data = pandas.read_csv("C:/Users/Sanskriti/Downloads/DSL-StrongPasswordData.csv")
subjects = data["subject"].unique()
print ("average EER for Manhattan filtered detector: ")
print(ManhattanFilteredDetector(subjects).evaluate())

average EER for Manhattan filtered detector: 
0.13441161913402994
